In [ ]:
import requests
from cocktail import *
import requests
from recipe import *
import os
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import movie_preprocessing
import helperfunctions

os.environ['ROOT_PATH'] = os.path.abspath(os.path.join("..",os.curdir))

current_directory = os.path.dirname(os.path.abspath(__file__))

SCRIPT_FOLDER = os.path.join(current_directory, 'data/scripts')  
FOOD_DATABASE = os.path.join(current_directory, 'database.txt')  

app = Flask(__name__)
CORS(app)

@app.route("/")
def home():
    return render_template('base.html', title="Movie Food Finder")

@app.route("/find-foods")
def find_foods():
    """API endpoint to find foods in a movie script"""
    movie_title = request.args.get('movie', '').strip()
    
    if not movie_title:
        return jsonify({"error": "Please enter a movie title"})
    
    script = helperfunctions.get_movie_script(movie_title, SCRIPT_FOLDER)
    script = helperfunctions.get_movie_script(movie_title, SCRIPT_FOLDER)
    if not script:
        return jsonify({"error": "Script not found"})
    
    script_tfidf = vectorizer.transform([script])
    script_projected = script_tfidf.dot(vt.T)
    script_projected = normalize(script_projected)
    
    # Cocktail SVD
    similarities = script_projected.dot(cocktail_vectors.T)
    top_indices = np.argsort(-similarities[0])[:3]
    svd_results = [cocktails[i] for i in top_indices]
    # if not svd_results:
    #     jaccard_scores = [
    #         (i, jaccard_similarity(script, " ".join(
    #             [c['strIngredient1'], c['strIngredient2'] ] # Add more ingredients as needed
    #         )))
    #         for i, c in enumerate(cocktails)
    #     ]
    #     top_jaccard = sorted(jaccard_scores, key=lambda x: -x[1])[:5]
    #     return jsonify([cocktails[i] for i, _ in top_jaccard])
    cleaned_results = [clean_cocktail_data(c) for c in svd_results]
    # print(cleaned_results)

    # Recipe SVD
    similarities = script_projected.dot(recipe_vectors.T)
    top_indices = np.argsort(-similarities[0])[:3]
    svd_results = [recipes[i] for i in top_indices]